In [1]:
//import $ivy.`org.typelevel::cats-core:2.1.1`
import $plugin.$ivy.`org.spire-math::kind-projector:0.9.9`

import $plugin.$                                          

In [2]:
case class Cont[W, A](run: (A => W) => W)

def pure[W, A]: A => Cont[W, A] = 
    a => Cont(f => f(a))

def runC[A]: Cont[A, A] => A = 
    _.run(identity)

def reset[W, A]: Cont[A, A] => Cont[W, A] = 
    runC[A] andThen pure[W, A]

def shift[W, A]: ((A => W) => Cont[W, W]) => Cont[W, A] = 
    f => Cont(f andThen runC)

defined class Cont
defined function pure
defined function runC
defined function reset
defined function shift

In [3]:
def bind[W, A, B](c: Cont[W, A])(f: A => Cont[W, B]): Cont[W, B] = 
    Cont((g: B => W) => {
        c.run((a: A) => 
            f(a).run(g))
    })

defined function bind

In [4]:
def map2[W, A](c1: Cont[W, A], c2: Cont[W, A])(f: (A, A) => A): Cont[W, A] = 
    Cont((g: A => W) => {
        c1.run((a1: A) => 
            c2.run((a2: A) => 
                g(f(a1,a2))))
    })

defined function map2

In [5]:

implicit class LiftA[W](c1: Cont[W, Int]){
    def *(c2: Cont[W, Int]): Cont[W, Int] = 
        map2(c1, c2)(_ * _)
    def +(c2: Cont[W, Int]): Cont[W, Int] = 
        map2(c1, c2)(_ + _)
    def -(c2: Cont[W, Int]): Cont[W, Int] = 
        map2(c1, c2)(_ - _)
}

defined class LiftA

In [6]:
def t2[W]: Cont[W, Int] = 
    reset(
        pure[Int, Int](3) + shift(
            (k: Int => Int) => pure(k(10)))) - 
    pure(1)

defined function t2

In [7]:
runC(t2[Int])

res6: Int = 12

In [8]:
def t3[W]: Cont[W, Int] = 
    reset(pure[Int, Int](2) * shift(k => pure(k(k(10))))) + pure(1)

defined function t3

In [9]:
runC(t3[Int])

res8: Int = 41

# How to discard delimited continuations

In [9]:
def t4[W] = // : Cont[W, String] = 
    reset(pure(2) * shift((k: Int => String) => pure[String, String]("abort")))

cmd9.sc:2: type mismatch;
 found   : ammonite.$sess.cmd4.wrapper.cmd1.Cont[String,Int]
 required: cmd9.this.cmd1.Cont[Any,Any]
Note: String <: Any, but class Cont is invariant in type W.
You may wish to define W as +W instead. (SLS 4.5)
Note: Int <: Any, but class Cont is invariant in type A.
You may wish to define A as +A instead. (SLS 4.5)
    reset(pure(2) * shift((k: Int => String) => pure[String, String]("abort")))
                  ^Compilation Failed

: 